# LSTM con embeddings

Modelo básico con los embeddings de fasttext

In [1]:
import pandas as pd
import csv
import numpy as np
import tensorflow as tf
import random

np.random.seed(2019)
tf.random.set_random_seed(2019)
random.seed(2019)

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_test = pd.read_table("../../../data/es/reference_es.tsv", header=None, 
                        names=["text", "HS", "TR", "AG"], quoting=csv.QUOTE_NONE)


text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]
text_test, y_test = df_test["text"], df_test["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500


Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 200000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)
X_test = tokenizer.texts_to_sequences(text_test)
max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)
X_test = pad_sequences(X_test, max_length)

Using TensorFlow backend.


Carguemos embeddings

In [3]:
import os
from glob import glob

path_to_embeddings = os.path.expanduser("/home/jmperez/WordVectors/")

print("Available embeddings: ", glob(os.path.join(path_to_embeddings, "*.vec")))

Available embeddings:  ['/home/jmperez/WordVectors/UBA_w5_200.vec', '/home/jmperez/WordVectors/wiki.es.vec', '/home/jmperez/WordVectors/UBA_w5_300.vec']


# Twitter Embeddings

In [4]:
import numpy as np

word_to_vec = {}

with open(os.path.join(path_to_embeddings, "UBA_w5_300.vec")) as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            vec = np.asarray(values[1:], dtype="float32")
        except:
            print(("*" * 80  + "\n")*3)
            print("Problema con la sig línea:")
            print(values[:10])
            word = values[1]
            vec = np.asarray(values[2:], dtype="float32")
        word_to_vec[word] = vec
        
embedding_size = len(word_to_vec["hola"])

********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '.', '-0.22232', '0.0052569', '0.47066', '0.13836', '0.15991', '0.19504', '0.00067885', '0.020299']
********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '...', '-0.11666', '-0.083768', '0.028919', '0.29973', '0.21017', '0.27808', '0.063251', '0.090223']
********************************************************************************
********************************************************************************
********************************************************************************

P

In [5]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    embedding_vector = word_to_vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [6]:
embedding_matrix.shape

(200000, 300)

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import CuDNNLSTM, Dropout, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(CuDNNLSTM(100, return_sequences=True))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 3s 616us/step - loss: 0.6452 - acc: 0.6369 - val_loss: 0.6090 - val_acc: 0.6680
Epoch 2/20
4500/4500 [==============================] - 1s 324us/step - loss: 0.5482 - acc: 0.7296 - val_loss: 0.5503 - val_acc: 0.7280
Epoch 3/20
4500/4500 [==============================] - 1s 327us/step - loss: 0.4957 - acc: 0.7698 - val_loss: 0.5484 - val_acc: 0.7260
Epoch 4/20
4500/4500 [==============================] - 1s 323us/step - loss: 0.4606 - acc: 0.7829 - val_loss: 0.5216 - val_acc: 0.7540
Epoch 5/20
4500/4500 [==============================] - 1s 326us/step - loss: 0.4397 - acc: 0.7969 - val_loss: 0.5207 - val_acc: 0.7360
Epoch 6/20
4500/4500 [==============================] - 1s 328us/step - loss: 0.4171 - acc: 0.8111 - val_loss: 0.5179 - val_acc: 0.7540
Epoch 7/20
4500/4500 [==============================] - 1s 331us/step - loss: 0.3986 - acc: 0.8244 - val_loss: 0.5125 - val_acc: 0.7520
E

In [9]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)



Evaluación sobre dev
500/500 [==============================] - 0s 105us/step
Loss           : 0.5589
Accuracy       : 0.7700
Precision(1)   : 0.7421
Precision(1)   : 0.7921
Precision(avg) : 0.7671

Recall(1)      : 0.7387
Recall(0)      : 0.7950
Recall(avg)    : 0.7669

F1(1)          : 0.7404
F1(0)          : 0.7935
F1(avg)        : 0.7670


Evaluación sobre test
1600/1600 [==============================] - 0s 111us/step
Loss           : 0.6933
Accuracy       : 0.6925
Precision(1)   : 0.6135
Precision(1)   : 0.7605
Precision(avg) : 0.6870

Recall(1)      : 0.6879
Recall(0)      : 0.6957
Recall(avg)    : 0.6918

F1(1)          : 0.6486
F1(0)          : 0.7267
F1(avg)        : 0.6876


## Bidirectional LSTM

In [13]:
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(CuDNNLSTM(100, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 14s 3ms/step - loss: 0.6167 - acc: 0.6598 - val_loss: 0.6379 - val_acc: 0.6540
Epoch 2/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.5100 - acc: 0.7529 - val_loss: 0.6359 - val_acc: 0.6320
Epoch 3/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.4582 - acc: 0.7882 - val_loss: 0.5076 - val_acc: 0.7520
Epoch 4/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.4246 - acc: 0.8071 - val_loss: 0.5161 - val_acc: 0.7440
Epoch 5/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.3954 - acc: 0.8198 - val_loss: 0.4887 - val_acc: 0.7520
Epoch 6/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.3765 - acc: 0.8313 - val_loss: 0.4759 - val_acc: 0.7820
Epoch 7/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.3531 - acc: 0.8438 - val_loss: 0.4841 - val_acc: 0.7700
Epoch 8/

In [10]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)



Evaluación sobre dev
500/500 [==============================] - 0s 124us/step
Loss           : 0.5589
Accuracy       : 0.7700
Precision(1)   : 0.7421
Precision(1)   : 0.7921
Precision(avg) : 0.7671

Recall(1)      : 0.7387
Recall(0)      : 0.7950
Recall(avg)    : 0.7669

F1(1)          : 0.7404
F1(0)          : 0.7935
F1(avg)        : 0.7670


Evaluación sobre test
1600/1600 [==============================] - 0s 115us/step
Loss           : 0.6933
Accuracy       : 0.6925
Precision(1)   : 0.6135
Precision(1)   : 0.7605
Precision(avg) : 0.6870

Recall(1)      : 0.6879
Recall(0)      : 0.6957
Recall(avg)    : 0.6918

F1(1)          : 0.6486
F1(0)          : 0.7267
F1(avg)        : 0.6876


## Probemos GRU + Conv

In [11]:
from keras.layers import CuDNNGRU

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(CuDNNGRU(100, return_sequences=True))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(1, activation='sigmoid'))

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 2s 431us/step - loss: 0.6085 - acc: 0.6700 - val_loss: 0.5627 - val_acc: 0.7120
Epoch 2/20
4500/4500 [==============================] - 1s 320us/step - loss: 0.5111 - acc: 0.7618 - val_loss: 0.5400 - val_acc: 0.7340
Epoch 3/20
4500/4500 [==============================] - 1s 325us/step - loss: 0.4651 - acc: 0.7836 - val_loss: 0.5369 - val_acc: 0.7500
Epoch 4/20
4500/4500 [==============================] - 1s 322us/step - loss: 0.4376 - acc: 0.7996 - val_loss: 0.5376 - val_acc: 0.7500
Epoch 5/20
4500/4500 [==============================] - 1s 326us/step - loss: 0.4189 - acc: 0.8060 - val_loss: 0.5361 - val_acc: 0.7500
Epoch 6/20
4500/4500 [==============================] - 1s 325us/step - loss: 0.3947 - acc: 0.8196 - val_loss: 0.5304 - val_acc: 0.7480
Epoch 7/20
4500/4500 [==============================] - 1s 321us/step - loss: 0.3914 - acc: 0.8280 - val_loss: 0.5391 - val_acc: 0.7340
E

In [12]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)



Evaluación sobre dev
500/500 [==============================] - 0s 133us/step
Loss           : 0.5554
Accuracy       : 0.7680
Precision(1)   : 0.7477
Precision(1)   : 0.7832
Precision(avg) : 0.7654

Recall(1)      : 0.7207
Recall(0)      : 0.8058
Recall(avg)    : 0.7632

F1(1)          : 0.7339
F1(0)          : 0.7943
F1(avg)        : 0.7641


Evaluación sobre test
1600/1600 [==============================] - 0s 119us/step
Loss           : 0.6419
Accuracy       : 0.7006
Precision(1)   : 0.6310
Precision(1)   : 0.7536
Precision(avg) : 0.6923

Recall(1)      : 0.6606
Recall(0)      : 0.7287
Recall(avg)    : 0.6947

F1(1)          : 0.6454
F1(0)          : 0.7409
F1(avg)        : 0.6932


# Conv + LSTM

In [13]:
from keras.layers import GRU, MaxPooling1D

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(CuDNNLSTM(100, return_sequences=False))
model.add(Dropout(0.55))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(1, activation='sigmoid'))
optimizer_args = {
    "lr": 0.001,
    "decay": 0.02
}

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 2s 423us/step - loss: 0.6508 - acc: 0.6364 - val_loss: 0.6120 - val_acc: 0.6840
Epoch 2/20
4500/4500 [==============================] - 1s 303us/step - loss: 0.5556 - acc: 0.7329 - val_loss: 0.5558 - val_acc: 0.7240
Epoch 3/20
4500/4500 [==============================] - 1s 304us/step - loss: 0.5024 - acc: 0.7676 - val_loss: 0.5431 - val_acc: 0.7360
Epoch 4/20
4500/4500 [==============================] - 1s 301us/step - loss: 0.4730 - acc: 0.7918 - val_loss: 0.5309 - val_acc: 0.7440
Epoch 5/20
4500/4500 [==============================] - 1s 299us/step - loss: 0.4466 - acc: 0.8051 - val_loss: 0.5254 - val_acc: 0.7620
Epoch 6/20
4500/4500 [==============================] - 1s 296us/step - loss: 0.4262 - acc: 0.8156 - val_loss: 0.5301 - val_acc: 0.7580
Epoch 7/20
4500/4500 [==============================] - 1s 299us/step - loss: 0.4101 - acc: 0.8233 - val_loss: 0.5185 - val_acc: 0.7660
E

In [14]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)



Evaluación sobre dev
500/500 [==============================] - 0s 106us/step
Loss           : 0.5597
Accuracy       : 0.7760
Precision(1)   : 0.7236
Precision(1)   : 0.8268
Precision(avg) : 0.7752

Recall(1)      : 0.8018
Recall(0)      : 0.7554
Recall(avg)    : 0.7786

F1(1)          : 0.7607
F1(0)          : 0.7895
F1(avg)        : 0.7751


Evaluación sobre test
1600/1600 [==============================] - 0s 107us/step
Loss           : 0.7538
Accuracy       : 0.6813
Precision(1)   : 0.5915
Precision(1)   : 0.7756
Precision(avg) : 0.6836

Recall(1)      : 0.7348
Recall(0)      : 0.6436
Recall(avg)    : 0.6892

F1(1)          : 0.6554
F1(0)          : 0.7035
F1(avg)        : 0.6794
